# Домашнее задание к Уроку 6

1. взять любой набор данных для бинарной классификации (можно скачать один из модельных с https://archive.ics.uci.edu/ml/datasets.php)
2. сделать feature engineering
3. обучить любой классификатор (какой вам нравится)
4. далее разделить ваш набор данных на два множества: P (positives) и U (unlabeled). Причем брать нужно не все положительные (класс 1) примеры, а только лишь часть
5. применить random negative sampling для построения классификатора в новых условиях
6. сравнить качество с решением из пункта 4 (построить отчет - таблицу метрик)
7. поэкспериментировать с долей P на шаге 6 (как будет меняться качество модели при уменьшении/увеличении размера P)

#### Загрузим необходимые библиотеки

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import recall_score, precision_score, roc_auc_score, accuracy_score, f1_score

In [3]:
def evaluate_results(y_test, y_predict):
    print('Classification results:')
    f1 = f1_score(y_test, y_predict)
    print("f1: %.2f%%" % (f1 * 100.0)) 
    roc = roc_auc_score(y_test, y_predict)
    print("roc: %.2f%%" % (roc * 100.0)) 
    rec = recall_score(y_test, y_predict, average='binary')
    print("recall: %.2f%%" % (rec * 100.0)) 
    prc = precision_score(y_test, y_predict, average='binary')
    print("precision: %.2f%%" % (prc * 100.0)) 

#### Описание датасета

Данные связаны с кампаниями прямого маркетинга португальского банковского учреждения.

Маркетинговые кампании были основаны на телефонных звонках. Часто требовалось более одного контакта с одним и тем же клиентом,
чтобы обкеспечить согласие на доступ к подписке на продукт (срочный банковский депозит).

**Датасет содержит следующтие признаки:**
    
   1 - age (numeric) - возраст
   
   2 - job : тип работы (categorical: "admin.","unknown","unemployed","management","housemaid","entrepreneur","student",
                                       "blue-collar","self-employed","retired","technician","services")
                                       
   3 - marital : семейное положение (categorical: "married","divorced","single";)
   
   
   4 - education (categorical: "unknown","secondary","primary","tertiary") - образование
   
   5 - default: (binary: "yes","no") - наличие кредита
   
   6 - balance: среднегодовой баланс в евро (numeric) 
   
   7 - housing: наличие ипотеки? (binary: "yes","no") 
   
   8 - loan: наличие потребительского кредита? (binary: "yes","no") 
   
   **Признаки связанные с последней маркетиновой компанией:**
   
   9 - contact: канал взаимодействия (categorical: "unknown","telephone","cellular") 
   
  10 - day: день месяца последнего контакта (numeric) 
  
  11 - month: месяц последнего контакта (categorical: "jan", "feb", "mar", ..., "nov", "dec") -  
  
  12 - duration: продолжительность последнего контакта, в секундах (numeric)
  
  **прочие признаки:**
  
  13 - campaign: Общее количество контактов в рамках компании для клиента
  
  14 - pdays: количество дней, прошедших с момента последнего контакта с клиентом из предыдущей кампании (numeric, -1 значит что с клиентом не контактировали в предыдущих компаниях)
  
  15 - previous: количество контактов с клиентом, до этой кампании (numeric)
  
  16 - poutcome: результат предыдущей маркетинговой кампании (categorical: "unknown","other","failure","success")

  **Целевой признак**:
  
  17 - y - подписал ли клиент срочный депозит? (binary: "yes","no")

#### Загрузим датасеты

In [4]:
df = pd.read_csv('bank-full.csv', sep=';')

In [5]:
df.head(3)

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no


In [6]:
df.shape

(45211, 17)

Посмотрим на cоотношение классов целевого признака:

In [7]:
df['y'].value_counts()

no     39922
yes     5289
Name: y, dtype: int64

Очевиден дисбаланс классов

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45211 entries, 0 to 45210
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        45211 non-null  int64 
 1   job        45211 non-null  object
 2   marital    45211 non-null  object
 3   education  45211 non-null  object
 4   default    45211 non-null  object
 5   balance    45211 non-null  int64 
 6   housing    45211 non-null  object
 7   loan       45211 non-null  object
 8   contact    45211 non-null  object
 9   day        45211 non-null  int64 
 10  month      45211 non-null  object
 11  duration   45211 non-null  int64 
 12  campaign   45211 non-null  int64 
 13  pdays      45211 non-null  int64 
 14  previous   45211 non-null  int64 
 15  poutcome   45211 non-null  object
 16  y          45211 non-null  object
dtypes: int64(7), object(10)
memory usage: 5.9+ MB


Пропуски отсутствуют

In [9]:
df.describe()

,age,balance,day,duration,campaign,pdays,previous
count,45211.000000,45211.000000,45211.000000,45211.000000,45211.000000,45211.000000,45211.000000
mean,40.936210,1362.272058,15.806419,258.163080,2.763841,40.197828,0.580323
std,10.618762,3044.765829,8.322476,257.527812,3.098021,100.128746,2.303441
min,18.000000,-8019.000000,1.000000,0.000000,1.000000,-1.000000,0.000000
25%,33.000000,72.000000,8.000000,103.000000,1.000000,-1.000000,0.000000
50%,39.000000,448.000000,16.000000,180.000000,2.000000,-1.000000,0.000000
75%,48.000000,1428.000000,21.000000,319.000000,3.000000,-1.000000,0.000000
max,95.000000,102127.000000,31.000000,4918.000000,63.000000,871.000000,275.000000


Сразу выполним преобразование целевого признака:

In [10]:
# Произведем мапинг на 0 и 1 для признака y
binary_dict = {'no': 0, 'yes': 1}
df['y'] = df['y'].map(binary_dict).astype(int)

Разабьем датасет на тренировочный и тестовый:

In [11]:
target = df['y']
X = df.iloc[:, :-1]

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, target, test_size=0.3, random_state=7)

In [13]:
X_train.shape

(31647, 16)

In [14]:
categorical_columns = ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'poutcome']
ohe_columns = ['job', 'marital', 'education', 'contact']
numeric_columns = ['age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous']

#### Обработка признаков

Сначала обработаем бинарные признаки:

In [15]:
# Произведем мапинг на 0 и 1 для признака default
X_train['default'] = X_train['default'].map(binary_dict).astype(int)
X_test['default'] = X_test['default'].map(binary_dict).astype(int)

# Произведем мапинг на 0 и 1 для признака housing
X_train['housing'] = X_train['housing'].map(binary_dict).astype(int)
X_test['housing'] = X_test['housing'].map(binary_dict).astype(int)

# Произведем мапинг на 0 и 1 для признака loan
X_train['loan'] = X_train['loan'].map(binary_dict).astype(int)
X_test['loan'] = X_test['loan'].map(binary_dict).astype(int)

# Для признака poutcome будем считать 1 только для success все остальное пометим как 0
poutcome_dict = {'unknown': 0, 'failure': 0, 'other': 0, 'success': 1}
X_train['poutcome'] = X_train['poutcome'].map(poutcome_dict).astype(int)
X_test['poutcome'] = X_test['poutcome'].map(poutcome_dict).astype(int)

Обработаем оставшиеся категориальные признаки:

In [16]:
# Для признака month наименование месяцев переведем в их цифровое обозначение
month_dict = {'jan': 1, 'feb': 2, 'mar': 3, 'apr': 4, 'may': 5, 'jun': 6,
              'jul': 7, 'aug': 8, 'sep': 9, 'oct': 10, 'nov': 11, 'dec': 12}
X_train['month'] = X_train['month'].map(month_dict).astype(int)
X_test['month'] = X_test['month'].map(month_dict).astype(int)

In [17]:
# Остальные категориальные признаси обработаем через OHE
X_train = pd.get_dummies(X_train, columns=ohe_columns)
X_test = pd.get_dummies(X_test, columns=ohe_columns)

#### Модель XGBoost

Обучим модель и расчитаем метрики качества

In [19]:
xgb_model = XGBClassifier()

xgb_model.fit(X_train, y_train)
y_predict = xgb_model.predict(X_test)

evaluate_results(y_test, y_predict)

Classification results:
f1: 54.43%
roc: 72.29%
recall: 48.31%
precision: 62.32%


In [20]:
# Создадим словарь который будет содержать метрики моделей
models_results = {
    'Model': [],
    'F-Score': [],
    'Precision': [],
    'Recall': []
}

In [21]:
models_results['Model'].append('XGB_no_PU')
models_results['F-Score'].append(f1_score(y_test, y_predict))
models_results['Precision'].append(precision_score(y_test, y_predict))
models_results['Recall'].append(recall_score(y_test, y_predict))

### random negative sampling

In [22]:
# Создадим функцию преобразования в PU и получения смешаной выборки из определенного процента записей датасета
def get_pu_samples(X_train, y_train, part):
    # получим индексы записей с позитивными значениями
    positiv_idx = y_train[y_train == 1].index.to_list()
    # Перемешаем индексы
    np.random.shuffle(positiv_idx)
    # Оставим тоько part процентов записей
    positiv_sample_idx = int(np.ceil(part * len(positiv_idx)))
    
    #print(f'Используем {positiv_sample_idx}/{len(positiv_idx)} как позитивные и сделаем оставшуюся чать не размечеными!')
    positiv_sample = positiv_idx[:positiv_sample_idx]
    
    X_train_pu = X_train.copy()
    X_train_pu['y'] = y_train
    
    # Создаем столбец для новой целевой переменной, где у нас два класса - P (1) и U (-1)
    X_train_pu['y_test'] = -1
    X_train_pu.loc[positiv_sample, 'y_test'] = 1
    #print('\nTarget variable:\n', X_train_pu.iloc[:,-1].value_counts())
    
    # Сформируем новый датасет для обучения
    X_train_pu = X_train_pu.sample(frac=1)
    negative_sample = X_train_pu[X_train_pu['y_test']==-1][:len(X_train_pu[X_train_pu['y_test']==1])]
    sample_test = X_train_pu[X_train_pu['y_test']==-1][len(X_train_pu[X_train_pu['y_test']==1]):]
    positiv_sample = X_train_pu[X_train_pu['y_test']==1]
    
    #print(f'\nРазмер негативной выборки: {negative_sample.shape}\nРазмер позитивной выборки: {positiv_sample.shape}')
    sample_train = pd.concat([negative_sample, positiv_sample]).sample(frac=1)
    return sample_train, sample_test

In [25]:
# Создадим функцию обучения модели
def pu_custom_xgb(simple_train, simple_test, models_results, part):
    x_train = sample_train.iloc[:,:-2]
    x_test = sample_test.iloc[:,:-2]
    y_train = sample_train.iloc[:,-2]
    y_test = sample_test.iloc[:,-2]
    
    xgb_model = XGBClassifier()
    xgb_model.fit(x_train, y_train)
    y_predict = xgb_model.predict(x_test)
    #evaluate_results(y_test, y_predict)
    
    # Внесем метрики полученной модели в словарь
    models_results['Model'].append('XGB_PU_' + str(part*100) + '%')
    models_results['F-Score'].append(f1_score(y_test, y_predict))
    models_results['Precision'].append(precision_score(y_test, y_predict))
    models_results['Recall'].append(recall_score(y_test, y_predict))

In [26]:
# Задаим спсок процентов pu выборок
part = [0.15, 0.25, 0.5, 0.75, 0.9]

# Обучим модели и расчитаем метрки для каждого из процентов
for percent in part:
    sample_train, sample_test = get_pu_samples(X_train, y_train, percent)
    pu_custom_xgb(sample_train, sample_test, models_results, percent)

In [27]:
# Выведем метрики полученые для заданых моделей
pd.DataFrame(data=models_results).sort_values('F-Score', ascending=False)

,Model,F-Score,Precision,Recall
0,XGB_no_PU,0.544253,0.623213,0.483051
1,XGB_PU_15.0%,0.475695,0.328981,0.858599
2,XGB_PU_25.0%,0.448976,0.300198,0.890114
3,XGB_PU_50.0%,0.392128,0.251030,0.895421
4,XGB_PU_75.0%,0.254784,0.149022,0.877672
5,XGB_PU_90.0%,0.119197,0.064007,0.865204


Вывод: Общая метрика качества F-score падает при использовании PU, но при этом с увеличением процента позитивных записей активно растет полнота при активном падении точности. При малых значениях процента PU выборки общая точность модели выше.